## Setup

In [20]:
import numpy as np
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.models import Sequential
from keras.optimizers import Adam, Nadam, SGD
from keras.preprocessing import image
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, Activation
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
x_train = x_train.reshape(x_train.shape[0], 1, 28, 28).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 1, 28, 28).astype('float32')
(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

((60000, 1, 28, 28), (60000,), (10000, 1, 28, 28), (10000,))

In [3]:
y_train[:5]

array([5, 0, 4, 1, 9], dtype=uint8)

In [4]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [5]:
y_train[:5]

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]])

In [6]:
mean_px = x_train.mean().astype(np.float32)
std_px = x_train.std().astype(np.float32)
def norm_input(x): return (x - mean_px) / std_px

## Linear Model

In [11]:
def get_lin_model_1():
    model = Sequential([
        Lambda(norm_input, input_shape = (1, 28, 28)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model
def get_lin_model():
    model = Sequential([
        Flatten(input_shape = (1, 28, 28)),
        Dense(10, activation='softmax')
    ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [46]:
lm = get_lin_model_1()

In [40]:
batch_size = 64
gen = image.ImageDataGenerator()
batches = gen.flow(x_train, y_train, batch_size=batch_size)
test_batches = gen.flow(x_test, y_test, batch_size=batch_size)

/root/anaconda3/lib/python3.6/site-packages/keras/preprocessing/image.py:855: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (60000, 1, 28, 28) (28 channels).
  ' (' + str(self.x.shape[channels_axis]) + ' channels).')
/root/anaconda3/lib/python3.6/site-packages/keras/preprocessing/image.py:855: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (10000, 1, 28, 28) (28 channels).
  ' (' + str(self.x.shape[channels_axis]) + ' channels).')


经测试samplewise_std_normalization=True时，与使用norm_input效果类似，但是运行效率差4倍,注意不要同时使用

In [18]:
lm.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=1,
                 validation_data=test_batches, validation_steps=test_batches.n//batch_size)

Epoch 1/1
938/937 [==============================] - 5s 5ms/step - loss: 0.3631 - acc: 0.8966 - val_loss: 0.2957 - val_acc: 0.9167


In [36]:
lm.optimizer.lr = 0.1

In [42]:
lm.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=1,
                 validation_data=test_batches, validation_steps=test_batches.n//batch_size)

Epoch 1/1
938/937 [==============================] - 11s 11ms/step - loss: 0.4304 - acc: 0.8708 - val_loss: 0.3028 - val_acc: 0.9157


In [13]:
lm.optimizer.lr = 0.01

In [49]:
!rm -rf ./Graph
tb_cb = TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=False, write_images=False)
lm.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=4,
                 validation_data=test_batches, validation_steps=test_batches.n//batch_size, callbacks=[tb_cb], workers=1)

Epoch 1/4
938/937 [==============================] - 6s 6ms/step - loss: 0.2527 - acc: 0.9299 - val_loss: 0.2803 - val_acc: 0.9241
Epoch 2/4
938/937 [==============================] - 6s 6ms/step - loss: 0.2513 - acc: 0.9292 - val_loss: 0.2907 - val_acc: 0.9206
Epoch 3/4
938/937 [==============================] - 3s 3ms/step - loss: 0.2506 - acc: 0.9305 - val_loss: 0.2837 - val_acc: 0.9240
Epoch 4/4
938/937 [==============================] - 6s 6ms/step - loss: 0.2492 - acc: 0.9304 - val_loss: 0.2846 - val_acc: 0.9209


## Single Dense Layer

In [70]:
def get_single_dense_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(1, 28, 28)),
        Flatten(),
        Dense(512, activation='softmax'),
        Dense(10, activation='softmax')
    ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [71]:
single_dense_model = get_single_dense_model()

In [72]:
single_dense_model.optimizer.lr = 0.1
single_dense_model.fit_generator(batches, steps_per_epoch=batches.n//batch_size,
                                 epochs=4, validation_data=test_batches, validation_steps=test_batches.n//batch_size)

Epoch 1/4
938/937 [==============================] - 14s 15ms/step - loss: 1.1450 - acc: 0.6397 - val_loss: 1.0715 - val_acc: 0.7065
Epoch 2/4
938/937 [==============================] - 13s 14ms/step - loss: 0.9254 - acc: 0.7151 - val_loss: 0.9265 - val_acc: 0.7120
Epoch 3/4
938/937 [==============================] - 13s 14ms/step - loss: 0.8712 - acc: 0.7162 - val_loss: 0.9473 - val_acc: 0.7164
Epoch 4/4
938/937 [==============================] - 13s 14ms/step - loss: 0.8445 - acc: 0.7259 - val_loss: 0.8214 - val_acc: 0.7283


In [23]:
single_dense_model.optimizer.lr=0.01
single_dense_model.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=10, validation_data=test_batches, validation_steps=test_batches.n//batch_size)

Epoch 1/10
938/937 [==============================] - 15s 16ms/step - loss: 0.2677 - acc: 0.9356 - val_loss: 0.2991 - val_acc: 0.9265
Epoch 2/10
938/937 [==============================] - 14s 15ms/step - loss: 0.2531 - acc: 0.9393 - val_loss: 0.2855 - val_acc: 0.9296
Epoch 3/10
938/937 [==============================] - 14s 15ms/step - loss: 0.2403 - acc: 0.9418 - val_loss: 0.2737 - val_acc: 0.9334
Epoch 4/10
938/937 [==============================] - 14s 15ms/step - loss: 0.2305 - acc: 0.9438 - val_loss: 0.2640 - val_acc: 0.9342
Epoch 5/10
938/937 [==============================] - 14s 15ms/step - loss: 0.2167 - acc: 0.9468 - val_loss: 0.2698 - val_acc: 0.9368
Epoch 6/10
938/937 [==============================] - 12s 12ms/step - loss: 0.2064 - acc: 0.9496 - val_loss: 0.2539 - val_acc: 0.9399
Epoch 7/10
938/937 [==============================] - 14s 15ms/step - loss: 0.1989 - acc: 0.9506 - val_loss: 0.2457 - val_acc: 0.9360
Epoch 8/10
938/937 [==============================] - 14s 15ms

## Vgg + Batchnorm + dropout + data augmentation

In [63]:
def get_vgg_improve():
    model = Sequential([
        Lambda(norm_input, input_shape=(1, 28, 28)),
        Conv2D(32, 3, 3, activation='relu', input_shape=(1, 28, 28)),
        BatchNormalization(axis=1),
        Conv2D(32, 3, 3, activation='relu'),
        MaxPooling2D(),
        BatchNormalization(axis=1),
        Conv2D(64, 3, 3, activation='relu'),
        BatchNormalization(axis=1),
        Conv2D(64, 3, 3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model
def get_model():
    model = Sequential([
        Lambda(norm_input, input_shape=(28,28,1)),
        ZeroPadding2D(),
        Convolution2D(28,3,3, activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [65]:
model = get_model()
model.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=4, validation_data=test_batches, validation_steps=test_batches.n//batch_size)

/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(28, (3, 3), activation="relu")`


Epoch 1/4


ValueError: Error when checking input: expected lambda_15_input to have shape (None, 28, 28, 1) but got array with shape (64, 1, 28, 28)

In [80]:
# Larger CNN for the MNIST Dataset
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]
def larger_model():
	# create model
	model = Sequential()
	model.add(Conv2D(30, (5, 5), input_shape=(1, 28, 28), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Conv2D(15, (3, 3), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dense(50, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [88]:
model = VGG16()
model.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=4, validation_data=test_batches, validation_steps=test_batches.n//batch_size)

RuntimeError: You must compile your model before using it.

In [81]:
# build the model
model = larger_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Large CNN Error: %.2f%%" % (100-scores[1]*100))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 35s 591us/step - loss: 0.3921 - acc: 0.8802 - val_loss: 0.0976 - val_acc: 0.9679
Epoch 2/10
60000/60000 [==============================] - 32s 534us/step - loss: 0.0956 - acc: 0.9704 - val_loss: 0.0578 - val_acc: 0.9806
Epoch 3/10
60000/60000 [==============================] - 31s 523us/step - loss: 0.0691 - acc: 0.9789 - val_loss: 0.0373 - val_acc: 0.9887
Epoch 4/10
60000/60000 [==============================] - 31s 518us/step - loss: 0.0561 - acc: 0.9827 - val_loss: 0.0315 - val_acc: 0.9893
Epoch 5/10
60000/60000 [==============================] - 31s 513us/step - loss: 0.0476 - acc: 0.9852 - val_loss: 0.0301 - val_acc: 0.9906
Epoch 6/10
60000/60000 [==============================] - 31s 524us/step - loss: 0.0429 - acc: 0.9864 - val_loss: 0.0281 - val_acc: 0.9903
Epoch 7/10
60000/60000 [==============================] - 31s 513us/step - loss: 0.0384 - acc: 0.9879 - val_loss: 0.

Large CNN Error: 0.82%


## RestNet

In [9]:
#-*- coding: UTF-8 -*-
"""
Environment: Keras2.0.5，Python2.7
Model: ResNet
"""

from __future__ import division
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Input, Activation, Dense, Flatten
from keras.layers.merge import add
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K
from keras.utils import plot_model
import six



def _handle_dim_ordering():
    global ROW_AXIS
    global COL_AXIS
    global CHANNEL_AXIS
    if K.image_dim_ordering() == 'tf':
        ROW_AXIS = 1
        COL_AXIS = 2
        CHANNEL_AXIS = 3
    else:
        CHANNEL_AXIS = 1
        ROW_AXIS = 2
        COL_AXIS = 3



def _get_block(identifier):
    if isinstance(identifier, six.string_types):
        res = globals().get(identifier)
        if not res:
            raise ValueError('Invalid {}'.format(identifier))
        return res
    return identifier



def _bn_relu(input):
    """
    Helper to build a BN -> relu block
    """

    norm = BatchNormalization(axis=CHANNEL_AXIS)(input)
    return Activation("relu")(norm)



def _conv_bn_relu(**conv_params):

    """
    Helper to build a conv -> BN -> relu block
    """

    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        conv = Conv2D(filters=filters, kernel_size=kernel_size,strides=strides, padding=padding,kernel_initializer=kernel_initializer,kernel_regularizer=kernel_regularizer)(input)
        return _bn_relu(conv)
    return f



def _bn_relu_conv(**conv_params):

    """
    Helper to build a BN -> relu -> conv block.
    This is an improved scheme proposed in http://arxiv.org/pdf/1603.05027v2.pdf
    """

    filters = conv_params["filters"]
    kernel_size = conv_params["kernel_size"]
    strides = conv_params.setdefault("strides", (1, 1))
    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")
    padding = conv_params.setdefault("padding", "same")
    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))

    def f(input):
        activation = _bn_relu(input)
        return Conv2D(filters=filters, kernel_size=kernel_size,strides=strides, padding=padding, kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer)(activation)
    return f



def _shortcut(input, residual):

    """
    Adds a shortcut between input and residual block and merges them with "sum"
    """
    # Expand channels of shortcut to match residual.
    # Stride appropriately to match residual (width, height)
    # Should be int if network architecture is correctly configured.

    input_shape = K.int_shape(input)
    residual_shape = K.int_shape(residual)
    stride_width = int(round(input_shape[ROW_AXIS] / residual_shape[ROW_AXIS]))
    stride_height = int(round(input_shape[COL_AXIS] / residual_shape[COL_AXIS]))
    equal_channels = input_shape[CHANNEL_AXIS] == residual_shape[CHANNEL_AXIS]

    shortcut = input

    # 1 X 1 conv if shape is different. Else identity.
    if stride_width > 1 or stride_height > 1 or not equal_channels:
        shortcut = Conv2D(filters=residual_shape[CHANNEL_AXIS], kernel_size=(1, 1), strides=(stride_width, stride_height), padding="valid", kernel_initializer="he_normal",kernel_regularizer=l2(0.0001))(input)
    return add([shortcut, residual])



def _residual_block(block_function, filters, repetitions, is_first_layer=False):

    """
    Builds a residual block with repeating bottleneck blocks.
    """

    def f(input):
        for i in range(repetitions):
            init_strides = (1, 1)
            if i == 0 and not is_first_layer:
                init_strides = (2, 2)
            input = block_function(filters=filters, init_strides=init_strides, is_first_block_of_first_layer=(is_first_layer and i == 0))(input)
        return input
    return f




def basic_block(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):

    """
    Basic 3 X 3 convolution blocks for use on resnets with layers <= 34.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf
    """

    def f(input):
        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv1 = Conv2D(filters=filters, kernel_size=(3, 3),strides=init_strides, padding="same", kernel_initializer="he_normal", kernel_regularizer=l2(1e-4))(input)
        else:
            conv1 = _bn_relu_conv(filters=filters, kernel_size=(3, 3), strides=init_strides)(input)
        residual = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv1)
        return _shortcut(input, residual)
    return f



def bottleneck(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):

    """
    Bottleneck architecture for > 34 layer resnet.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf
    Returns:
        A final conv layer of filters * 4
    """

    def f(input):
        if is_first_block_of_first_layer:
            # don't repeat bn->relu since we just did bn->relu->maxpool
            conv_1_1 = Conv2D(filters=filters, kernel_size=(1, 1), strides=init_strides, padding="same", kernel_initializer="he_normal", kernel_regularizer=l2(1e-4))(input)
        else:
            conv_1_1 = _bn_relu_conv(filters=filters, kernel_size=(1, 1), strides=init_strides)(input)

        conv_3_3 = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv_1_1)
        residual = _bn_relu_conv(filters=filters * 4, kernel_size=(1, 1))(conv_3_3)
        return _shortcut(input, residual)
    return f



class ResnetBuilder(object):
    @staticmethod
    def build(input_shape, num_outputs, block_fn, repetitions):
        """
        Builds a custom ResNet like architecture.
        Args:
            input_shape: The input shape in the form (nb_channels, nb_rows, nb_cols)
            num_outputs: The number of outputs at final softmax layer
            block_fn: The block function to use. This is either `basic_block` or `bottleneck`.The original paper used basic_block for layers < 50
            repetitions: Number of repetitions of various block units.At each block unit, the number of filters are doubled and the input size is halved
        Returns:
            The keras `Model`.
        """

        _handle_dim_ordering()

        if len(input_shape) != 3:
            raise Exception("Input shape should be a tuple (nb_channels, nb_rows, nb_cols)")

        # Permute dimension order if necessary
        if K.image_dim_ordering() == 'tf':
            input_shape = (input_shape[0], input_shape[1], input_shape[2])

        # Load function from str if needed.
        block_fn = _get_block(block_fn)

        input = Input(shape=input_shape)
        conv1 = _conv_bn_relu(filters=64, kernel_size=(7, 7), strides=(2, 2))(input)
        pool1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(conv1)

        block = pool1
        filters = 64
        for i, r in enumerate(repetitions):
            block = _residual_block(block_fn, filters=filters, repetitions=r, is_first_layer=(i == 0))(block)
            filters *= 2

        # Last activation
        block = _bn_relu(block)

        # Classifier block
        block_shape = K.int_shape(block)
        pool2 = AveragePooling2D(pool_size=(block_shape[ROW_AXIS], block_shape[COL_AXIS]), strides=(1, 1))(block)
        flatten1 = Flatten()(pool2)
        dense = Dense(units=num_outputs, kernel_initializer="he_normal", activation="softmax")(flatten1)

        model = Model(inputs=input, outputs=dense)
        return model


    @staticmethod
    def build_resnet_18(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [2, 2, 2, 2])

    @staticmethod
    def build_resnet_34(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [3, 4, 6, 3])

    @staticmethod
    def build_resnet_50(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 6, 3])

    @staticmethod
    def build_resnet_101(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 23, 3])

    @staticmethod
    def build_resnet_152(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 8, 36, 3])




model_18=ResnetBuilder.build_resnet_18((1, 28, 28), 10)
#model.summary()

# Save a PNG of the Model Build
#plot_model(model,to_file='ResNet.png')

model_18.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='sgd',loss='categorical_crossentropy')
model_18.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=1,
             validation_data=test_batches, validation_steps=test_batches.n//batch_size)
model_18.optimizer.lr=0.01
model_18.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=1,
             validation_data=test_batches, validation_steps=test_batches.n//batch_size)    
model_18.optimizer.lr=0.001
model_18.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=1,
             validation_data=test_batches, validation_steps=test_batches.n//batch_size)      

'\nEnvironment: Keras2.0.5，Python2.7\nModel: ResNet\n'

Epoch 1/1
118/117 [==============================] - 424s 4s/step - loss: 0.4889 - acc: 0.9491 - val_loss: 0.4442 - val_acc: 0.9402


Epoch 1/1
118/117 [==============================] - 380s 3s/step - loss: 0.2872 - acc: 0.9847 - val_loss: 0.3024 - val_acc: 0.9732


Epoch 1/1
118/117 [==============================] - 383s 3s/step - loss: 0.2341 - acc: 0.9899 - val_loss: 0.2537 - val_acc: 0.9785


In [27]:
batch_size = 64
gen = image.ImageDataGenerator(samplewise_std_normalization=True)
batches = gen.flow(x_train, y_train, batch_size=batch_size)
test_batches = gen.flow(x_test, y_test, batch_size=batch_size)
model_18.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=1,
             validation_data=test_batches, validation_steps=test_batches.n//batch_size)

/root/anaconda3/lib/python3.6/site-packages/keras/preprocessing/image.py:855: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (60000, 1, 28, 28) (28 channels).
  ' (' + str(self.x.shape[channels_axis]) + ' channels).')
/root/anaconda3/lib/python3.6/site-packages/keras/preprocessing/image.py:855: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (10000, 1, 28, 28) (28 channels).
  ' (' + str(self.x.shape[channels_axis]) + ' channels).')


In [ ]:
# Create a Keras Model
model_34=ResnetBuilder.build_resnet_34((1, 28, 28), 10)
#model.summary()

# Save a PNG of the Model Build
#plot_model(model,to_file='ResNet.png')

model_34.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='sgd',loss='categorical_crossentropy')
model_34.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=1,
             validation_data=test_batches, validation_steps=test_batches.n//batch_size)
model_34.optimizer.lr=0.01
model_34.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=1,
             validation_data=test_batches, validation_steps=test_batches.n//batch_size)    
model_34.optimizer.lr=0.001
model_34.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=1,
             validation_data=test_batches, validation_steps=test_batches.n//batch_size)      

In [50]:
# Create a Keras Model
model_50=ResnetBuilder.build_resnet_50((1, 28, 28), 10)
#model.summary()

# Save a PNG of the Model Build
#plot_model(model,to_file='ResNet.png')

model_50.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='sgd',loss='categorical_crossentropy')
# print('Model Compiled')
model_50.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=1,
             validation_data=test_batches, validation_steps=test_batches.n//batch_size)
model_50.optimizer.lr=0.01
model_50.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=1,
             validation_data=test_batches, validation_steps=test_batches.n//batch_size)    
model_50.optimizer.lr=0.001
model_50.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=1,
             validation_data=test_batches, validation_steps=test_batches.n//batch_size)  

Epoch 1/1
938/937 [==============================] - 1048s 1s/step - loss: 2.1207 - acc: 0.9431 - val_loss: 1.0765 - val_acc: 0.9472


Epoch 1/1
938/937 [==============================] - 955s 1s/step - loss: 0.6889 - acc: 0.9709 - val_loss: 0.6281 - val_acc: 0.9538


Epoch 1/1
938/937 [==============================] - 985s 1s/step - loss: 0.4121 - acc: 0.9739 - val_loss: 0.3865 - val_acc: 0.9625


主要试试steps_per_epoch能不能很快的打印日志，验证后发现可以提前终止epoch，但是旧版本的keras有bug，无法提前终止

In [ ]:
# Create a Keras Model
model_50=ResnetBuilder.build_resnet_50((1, 28, 28), 10)
#model.summary()

# Save a PNG of the Model Build
#plot_model(model,to_file='ResNet.png')

model_50.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='sgd',loss='categorical_crossentropy')
# print('Model Compiled')
model_50.fit_generator(batches, steps_per_epoch=100, epochs=1,
             validation_data=test_batches, validation_steps=test_batches.n//batch_size)
model_50.optimizer.lr=0.01
model_50.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=1,
             validation_data=test_batches, validation_steps=100)    
model_50.optimizer.lr=0.001
model_50.fit_generator(batches, steps_per_epoch=100, epochs=1,
             validation_data=test_batches, validation_steps=100)  

Epoch 1/1
938/100 [=========================================================================================================================================================================================================================================================================================] - 1061s 1s/step - loss: 1.9315 - acc: 0.9550 - val_loss: 1.0314 - val_acc: 0.9698


Epoch 1/1
938/937 [==============================] - 1050s 1s/step - loss: 0.7116 - acc: 0.9713 - val_loss: 0.4949 - val_acc: 0.9760


Epoch 1/1
938/100 [=========================================================================================================================================================================================================================================================================================] - 998s 1s/step - loss: 0.4279 - acc: 0.9743 - val_loss: 0.3690 - val_acc: 0.9719


In [ ]:
# Create a Keras Model
model_101=ResnetBuilder.build_resnet_101((1, 28, 28), 10)
#model.summary()

# Save a PNG of the Model Build
#plot_model(model,to_file='ResNet.png')

model_101.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='sgd',loss='categorical_crossentropy')
print('Model Compiled')
model_101.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=1,
             validation_data=test_batches, validation_steps=test_batches.n//batch_size)
model_101.optimizer.lr=0.01
model_101.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=1,
             validation_data=test_batches, validation_steps=test_batches.n//batch_size)    
model_101.optimizer.lr=0.001
model_101.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=1,
             validation_data=test_batches, validation_steps=test_batches.n//batch_size) 

Model Compiled
Epoch 1/1
938/937 [==============================] - 1913s 2s/step - loss: 3.5878 - acc: 0.9420 - val_loss: 1.2898 - val_acc: 0.9719


Epoch 1/1
829/937 [=========================>....] - ETA: 3:18 - loss: 0.9388 - acc: 0.9659

In [ ]:
# Create a Keras Model
model_152=ResnetBuilder.build_resnet_152((1, 28, 28), 10)
#model.summary()

# Save a PNG of the Model Build
#plot_model(model,to_file='ResNet.png')

model_152.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='sgd',loss='categorical_crossentropy')
# print('Model Compiled')
model_152.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=1,
             validation_data=test_batches, validation_steps=test_batches.n//batch_size)
model_152.optimizer.lr=0.01
model_152.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=1,
             validation_data=test_batches, validation_steps=test_batches.n//batch_size)    
model_152.optimizer.lr=0.001
model_152.fit_generator(batches, steps_per_epoch=batches.n//batch_size, epochs=1,
             validation_data=test_batches, validation_steps=test_batches.n//batch_size) 